In [1]:
# First lets clear all previous python imports and variables by resetting the python kernel.
%reset -f

#De nodige packages..
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
import os
import time
from IPython.display import display, IFrame
import folium

#De nodige data: geo_path is hier de geojson van Boone, df de dataset van Dagevos
geo_path = r'https://github.com/joostbouten/assignments/raw/master/Gemeentegrenzen_2016_zonder_water_simplified_wgs84.geojson'
url = 'https://github.com/joostbouten/assignments/raw/master/datasetAEA.dta'
df = pd.read_stata(url)

%matplotlib inline

In [2]:
df = df[df['Year']==2017] #Ik gebruik alleen de meest recente data

In [3]:
#Meijerijstad is een nieuwe gemeente uit deze drie oude gemeentes, ik zorg hier dat 2017 data van meijerijstad op alle 3 polygons geplot kan worden.
veghel = ['GM1948', 'Veghel',10.3851394057056,0.050411537,0.012248361,0,0.034707335,0.001930953,0.004052873,0.896648942,0,0,1948,'GM1948','Meierijstad',7.8,0,26.5,73.5,69.9,1.8,0.9,0.1,0.8,0.8,0.0,0.6,0.4,0.0,'',2017, 'GM1948', 'CR35','Noordoost-Noord-Brabant','PV30','Noord-Brabant',50.7,52.7,55.2,61.3,36.4,56.1,53.2,53.8,52.8,56.2,47.5,51.2,77.5,72.8,44.7,38.2,61.4,43.3,49.2,50.7,35.5,50.5,30.4,55,48.7,60.9,35.3,76.2,63.9,62.5,62.4,56.2,71.5,49,18.8,42.6,0,75.1,67,69.3,68.1,59.3,29.9,66.6,39.8,29.5,62.4,39.6,50.7,77.3,52.5,48.7,41.2,62,58,66,64.4,49.4,23.1,42.1,58.6,61.3,50.4,62.9,41.5,62.6,30,100,47.4,21.6,23.3,75.6,57.5,24,19.9,78.8,79.7,77.9,82.2,80.8,60.2,91.3,70.5,79.9,77.6,43.4,58,81.7,51.2,90.4,100,47.2,44,43.6,54.1,34.8,0,43.4,62.1,23.2,53.2,35.3,50.3,57.3,83.6,27.3,47.4,71.2,63.6,49.9,27.8,59.9,31.3,77,52.4,68.2,63.7,31.6,37.5,43.9,42.1,20.9,35.1,24.9,62.4,87.8,25,52.6,24,53.1,14.3,14.1]
schijndel = ['GM1948', 'Schijndel',10.3851394057056,0.050411537,0.012248361,0,0.034707335,0.001930953,0.004052873,0.896648942,0,0,1948,'GM1948','Meierijstad',7.8,0,26.5,73.5,69.9,1.8,0.9,0.1,0.8,0.8,0.0,0.6,0.4,0.0,'',2017, 'GM1948', 'CR35','Noordoost-Noord-Brabant','PV30','Noord-Brabant',50.7,52.7,55.2,61.3,36.4,56.1,53.2,53.8,52.8,56.2,47.5,51.2,77.5,72.8,44.7,38.2,61.4,43.3,49.2,50.7,35.5,50.5,30.4,55,48.7,60.9,35.3,76.2,63.9,62.5,62.4,56.2,71.5,49,18.8,42.6,0,75.1,67,69.3,68.1,59.3,29.9,66.6,39.8,29.5,62.4,39.6,50.7,77.3,52.5,48.7,41.2,62,58,66,64.4,49.4,23.1,42.1,58.6,61.3,50.4,62.9,41.5,62.6,30,100,47.4,21.6,23.3,75.6,57.5,24,19.9,78.8,79.7,77.9,82.2,80.8,60.2,91.3,70.5,79.9,77.6,43.4,58,81.7,51.2,90.4,100,47.2,44,43.6,54.1,34.8,0,43.4,62.1,23.2,53.2,35.3,50.3,57.3,83.6,27.3,47.4,71.2,63.6,49.9,27.8,59.9,31.3,77,52.4,68.2,63.7,31.6,37.5,43.9,42.1,20.9,35.1,24.9,62.4,87.8,25,52.6,24,53.1,14.3,14.1]
stoedenrode = ['GM1948', 'Sint-Oedenrode',10.3851394057056,0.050411537,0.012248361,0,0.034707335,0.001930953,0.004052873,0.896648942,0,0,1948,'GM1948','Meierijstad',7.8,0,26.5,73.5,69.9,1.8,0.9,0.1,0.8,0.8,0.0,0.6,0.4,0.0,'',2017, 'GM1948', 'CR35','Noordoost-Noord-Brabant','PV30','Noord-Brabant',50.7,52.7,55.2,61.3,36.4,56.1,53.2,53.8,52.8,56.2,47.5,51.2,77.5,72.8,44.7,38.2,61.4,43.3,49.2,50.7,35.5,50.5,30.4,55,48.7,60.9,35.3,76.2,63.9,62.5,62.4,56.2,71.5,49,18.8,42.6,0,75.1,67,69.3,68.1,59.3,29.9,66.6,39.8,29.5,62.4,39.6,50.7,77.3,52.5,48.7,41.2,62,58,66,64.4,49.4,23.1,42.1,58.6,61.3,50.4,62.9,41.5,62.6,30,100,47.4,21.6,23.3,75.6,57.5,24,19.9,78.8,79.7,77.9,82.2,80.8,60.2,91.3,70.5,79.9,77.6,43.4,58,81.7,51.2,90.4,100,47.2,44,43.6,54.1,34.8,0,43.4,62.1,23.2,53.2,35.3,50.3,57.3,83.6,27.3,47.4,71.2,63.6,49.9,27.8,59.9,31.3,77,52.4,68.2,63.7,31.6,37.5,43.9,42.1,20.9,35.1,24.9,62.4,87.8,25,52.6,24,53.1,14.3,14.1]

In [4]:
columns = list(df.columns)
df_Meierij = pd.DataFrame([veghel], columns=columns).append(pd.DataFrame([schijndel], columns=columns)).append(pd.DataFrame([stoedenrode], columns=columns)) 

In [5]:
df = df.append(df_Meierij)

In [6]:
#Hier zorg ik dat namen in de geojson overeen komen met namen in de dataset
df.Municipalityname = df.Municipalityname.replace('S\xc3\x83\xc2\xbadwest-Frysl\xc3\x83\xc2\xa2n','Sudwest Fryslan')
df.Municipalityname = df.Municipalityname.replace('Bergen (L.)','Bergen (L)')
df.Municipalityname = df.Municipalityname.replace('Bergen (NH.)','Bergen (NH)')
df['municipality']=df['Municipalityname']
df['NAP'] =  df['AveragegroundlevelmNAP'] >= 0

In [7]:
#Hier definieer ik een functie die de kolomnaam en legendanaam als input nemen voor de plot

fn='map.html'

def NLMAP(colname,legendname):
        ref_map = folium.Map(
        location=[52.139177, 5.327108], # This will center the view on the world map where the Netherlands is located
        tiles='Mapbox Bright',          # This creates a base map and in this case its the Mapbox Bright basemap
        zoom_start=8, max_zoom=8)                   # This will zoom in on the center of view to get the Netherlands in full frame
        
        ref_map.choropleth(
        # This is the path to the geojson file that contains all the municipality shapes and locations
            geo_data=geo_path,
        # We will use the p_results dataframe for the choropleth mapping
            data=df,
        # municipality will be used for the mapping key and expenditure for its value
            columns=['municipality', colname],
        # Use GM_NAAM (short for municipality name) as keys for colormapping
            key_on='feature.properties.GM_NAAM',
        # We are going to use a color map from yellow to green
            fill_color='YlOrRd',
        # This gives municipality shapes some opacity so that we can still see the background
            fill_opacity=0.7,
        # This gives the lines around the municipality shapes some opacity so that they don't stand out too much
            line_opacity=0.2,
        # The legend
            legend_name=legendname)
        
        ref_map.save(fn)              # This will save the map in a HTML format
        display(IFrame(fn, 850,850)) # This displays the map in an Iframe


In [8]:
#Een voorbeeld van wat de functie doet:
NLMAP('Floods','Flood indicator') #Tussen de haakjes kan elke kolomnaam & legendanaam ingevuld worden

In [9]:
#Dit zorgt ervoor dat de plot geopend wordt op Firefox en dat er een screenshot gemaakt wordt.
#Hiervoor dient wel een webdriver voor geinstalleerd te zijn.
#De screenshot wordt in de working directory opgeslagen als 'map.png'

tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
#Open a browser window...
browser = webdriver.Firefox()
browser.set_window_size(750, 1850)
#..that displays the map...
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(5)
#Grab the screenshot
browser.save_screenshot('map.png')
#Close the browser
browser.quit()